In [11]:
import os
import glob
import pandas as pd
import numpy as np
from datetime import datetime


In [12]:
BASE_PATH = r"C:\Users\Administrator\LIS\Rahul\Colgate_Task"


In [13]:
NPG_MASTER_PATH     = os.path.join(BASE_PATH, "Master_NPG_SOS.xlsx")
STORE_MASTER_PATH   = os.path.join(BASE_PATH, "Master_Store.xlsx")
PRODUCT_MASTER_PATH = os.path.join(BASE_PATH, "Master_Product.xlsx")

SOS_FOLDER_PATH     = os.path.join(BASE_PATH, "SOS.csv")
# ASSET_PATH          = os.path.join(BASE_PATH, "Assets.csv")
PLANOGRAM_PATH      = os.path.join(BASE_PATH, "Plano.csv")
ROUTE_PATH          = os.path.join(BASE_PATH, "ROUTE.csv")
OSA_PATH            = os.path.join(BASE_PATH, "OSA.csv")

OUTPUT_PATH         = os.path.join(BASE_PATH, "Output")
os.makedirs(OUTPUT_PATH, exist_ok=True)


In [14]:
NPG_Data = pd.read_excel(NPG_MASTER_PATH)
Store_Master = pd.read_excel(STORE_MASTER_PATH)
Product_Master = pd.read_excel(PRODUCT_MASTER_PATH)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Administrator\\LIS\\Rahul\\Colgate_Task\\Master_NPG_SOS.xlsx'

In [ ]:
NPG_Data = NPG_Data.drop_duplicates(subset=["NPG Code"])
NPG_Data1 = NPG_Data[["NPG Code", "Category", "Brand"]]


In [ ]:
PD = Product_Master[
    ["LIS ProductCode", "ProductName", "MRP", "SubCategory"]
]


In [ ]:
Store_Master = Store_Master[
    [
        "New_DT_Store_Code",
        "LIS Store Code",
        "Store Name",
        "New_Amazing_EMP_ID",
        "LIS User Id",
        "Employee Name"
    ]
]


In [ ]:
sos_files = glob.glob(os.path.join(SOS_FOLDER_PATH, "*.csv"))

sos_list = []
for f in sos_files:
    df = pd.read_csv(f)
    sos_list.append(df)

sos_raw = pd.concat(sos_list, ignore_index=True)


In [ ]:
sos = sos_raw[
    [
        "Store Code",
        "Store Name",
        "Route Date",
        "NPG Code",
        "NPG Description",
        "Face On",
        "End on",
        "Stand On"
    ]
]


In [ ]:
sos = sos.merge(NPG_Data1, on="NPG Code", how="left")


In [ ]:
sos["Brands"] = np.where(sos["Brand"] == "Colgate", "Colgate", "Other")
sos.drop(columns=["Brand"], inplace=True)


In [ ]:
sos = sos.merge(
    Store_Master,
    left_on="Store Code",
    right_on="New_DT_Store_Code",
    how="left"
)


In [ ]:
sos["Total"] = (
    sos["Face On"] * 1 +
    sos["End on"] * 0.33 +
    sos["Stand On"] * 0.33
)

SOS_Data = sos.dropna(subset=["Category"])


In [ ]:
# pv = pd.read_csv(ASSET_PATH)

# pv1 = pv.merge(
#     Store_Master,
#     left_on="Store Code",
#     right_on="New_DT_Store_Code",
#     how="left"
# )

# pv2 = pv1[~pv1["LIS_Store_Code"].isna()]


In [ ]:
# pv2 = pv2[
#     [
#         "Route Date",
#         "New_Amazing_EMP_ID",
#         "LIS User Id",
#         "Employee Name",
#         "Store Code",
#         "LIS_Store_Code",
#         "Asset Code",
#         "Asset Description",
#         "Asset Availability"
#     ]
# ]

# pv2["Asset Availability"] = np.where(
#     pv2["Asset Availability"] == "Y", "Yes", "No"
# )


In [ ]:
plano = pd.read_csv(PLANOGRAM_PATH)

plano = plano.merge(
    Store_Master,
    left_on="Store Code",
    right_on="New_DT_Store_Code",
    how="left"
)

plano = plano[~plano["LIS User Id"].isna()]


In [ ]:
route = pd.read_csv(ROUTE_PATH)

route["Check In Date"]  = pd.to_datetime(route["Check In Date"], errors="coerce")
route["Check Out Date"] = pd.to_datetime(route["Check Out Date"], errors="coerce")

route["Time Duration"] = (
    route["Check Out Date"] - route["Check In Date"]
).dt.total_seconds() / 60


In [ ]:
permission = []
attendance = []

for _, row in route.iterrows():
    if row["Visit status"] == "Y":
        if row["Store status"] in ["Permission not granted", "Store closed", "-"]:
            permission.append("N")
            attendance.append("Permission")
        else:
            permission.append("Y")
            attendance.append("Present")
    else:
        permission.append("N")
        attendance.append("Absent")

route["Permission"] = permission
route["Attendance Type"] = attendance


In [ ]:
final = route.merge(
    Store_Master,
    left_on="Store Code",
    right_on="New_DT_Store_Code",
    how="left"
)

final = final.drop_duplicates()


In [ ]:
osa = pd.read_csv(OSA_PATH)

osa1 = osa.merge(
    Store_Master,
    left_on="Store Code",
    right_on="New_DT_Store_Code",
    how="left"
)

osa4 = osa1[~osa1["LIS_Store_Code"].isna()]


In [ ]:
SOS_Data.to_csv(os.path.join(OUTPUT_PATH, "SOS_Data.csv"), index=False)
# pv2.to_csv(os.path.join(OUTPUT_PATH, "Assets.csv"), index=False)
plano.to_csv(os.path.join(OUTPUT_PATH, "Planogram.csv"), index=False)
final.to_csv(os.path.join(OUTPUT_PATH, "Route.csv"), index=False)
osa4.to_csv(os.path.join(OUTPUT_PATH, "OSA.csv"), index=False)
